# Modelo de IA: Predicción de Empleabilidad de Graduados
Este notebook está diseñado para principiantes. Paso a paso aprenderemos a:

1. Cargar un dataset (descargado de Kaggle).
2. Explorar y limpiar los datos.
3. Preparar los datos para modelado (codificación, división train/test).
4. Entrenar un modelo básico (Random Forest) y evaluar resultados.
5. Visualizar resultados e interpretar la importancia de las variables.

🔸 **Instrucciones:** Descarga el archivo CSV del dataset de Kaggle (`Graduate_Employability.csv`) y súbelo a la misma carpeta donde ejecutarás este notebook (o móntalo en Google Colab).

🔸 El notebook incluye celdas marcadas con `# TODO` para que ustedes (los estudiantes) las completen.

In [ ]:
# 1) Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

%matplotlib inline

print('Librerías importadas correctamente')

## 2) Cargar el dataset
Carga el archivo CSV (asegúrate de que el nombre del archivo coincide con el que subiste).

In [ ]:
# Cambia la ruta si es necesario
csv_path = 'CollegePlacement.csv'

try:
    data = pd.read_csv(csv_path)
    print('Dataset cargado con éxito. Filas y columnas:', data.shape)
    display(data.head())
except FileNotFoundError:
    print(f'No se encontró el archivo {csv_path}. Sube el CSV y vuelve a ejecutar esta celda.')

## 3) Exploración básica de los datos
Veamos información general, tipos de datos y valores faltantes.

In [ ]:
# Información general
if 'data' in globals():
    display(data.info())
    display(data.describe(include='all').T)
    print('\nValores nulos por columna:')
    display(data.isnull().sum().sort_values(ascending=False).head(20))
else:
    print('Carga el dataset primero.')

### 3.1) Visualizaciones simples
Hagamos algunos gráficos para conocer la distribución de variables.

In [ ]:
if 'data' in globals():
    # Mostrar el conteo de una columna categórica si existe
    possible_cat_cols = data.select_dtypes(include=['object']).columns.tolist()
    print('Columnas categóricas detectadas:', possible_cat_cols)

    # Histograma de una variable numérica ejemplo
    num_cols = data.select_dtypes(include=[np.number]).columns.tolist()
    if len(num_cols) > 0:
        col = num_cols[0]
        plt.figure(figsize=(8,4))
        sns.histplot(data[col].dropna(), kde=True)
        plt.title(f'Histograma de {col}')
        plt.show()
else:
    print('Carga el dataset primero.')

## 4) Limpieza y preparación básica
Para mantener la clase simple vamos a: eliminar filas con valores nulos (opcional) y codificar variables categóricas.

In [ ]:
# 4.1) Eliminar filas con valores nulos (opcional para principiantes)
if 'data' in globals():
    print('Antes, shape =', data.shape)
    data_clean = data.dropna()
    print('Después de dropna(), shape =', data_clean.shape)
else:
    print('Carga el dataset primero.')

In [ ]:
# 4.2) Codificar variables categóricas con LabelEncoder
if 'data_clean' in globals():
    data_enc = data_clean.copy()
    le = LabelEncoder()
    cat_cols = data_enc.select_dtypes(include=['object']).columns.tolist()
    print('Columnas a codificar:', cat_cols)
    for c in cat_cols:
        try:
            data_enc[c] = le.fit_transform(data_enc[c])
        except Exception as e:
            print('No se pudo codificar', c, e)
    display(data_enc.head())
else:
    print('Asegúrate de tener data_clean')

## 5) Seleccionar variable objetivo y características
Buscamos predecir una columna de empleabilidad. Identifica el nombre correcto de la columna objetivo en tu dataset.

En muchos datasets puede llamarse `EmploymentStatus`, `Employed`, `Target`, etc.

In [ ]:
# 5.1) Revisa columnas y elige objetivo
if 'data_enc' in globals():
    print('Columnas disponibles:')
    display(pd.DataFrame({'columns': data_enc.columns}))
    # TODO: Si la columna objetivo tiene un nombre distinto, cámbialo aquí
    target_column = 'EmploymentStatus'  # <-- EDITA si tu dataset usa otro nombre
    if target_column not in data_enc.columns:
        print('\nNo se encontró la columna objetivo:', target_column)
        print('Por favor revisa el nombre y edítalo en la celda.')
    else:
        X = data_enc.drop(target_column, axis=1)
        y = data_enc[target_column]
        print('\nCaracterísticas (X) y objetivo (y) definidas con éxito')
else:
    print('Asegúrate de ejecutar las celdas anteriores primero.')

## 6) División de datos: entrenamiento y prueba
Dividimos los datos para evaluar correctamente el modelo (train/test split).

In [ ]:
if 'X' in globals() and 'y' in globals():
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    print('Tamaños:')
    print('X_train', X_train.shape, 'X_test', X_test.shape)
else:
    print('Define X e y en la celda anterior.')

## 7) Entrenar un modelo básico (Random Forest)
Usamos RandomForestClassifier porque funciona bien sin demasiada parametrización.
Explica a los estudiantes que es un conjunto de árboles de decisión.

In [ ]:
if 'X_train' in globals():
    model = RandomForestClassifier(random_state=42)
    model.fit(X_train, y_train)
    print('Modelo entrenado correctamente')
else:
    print('Asegúrate de ejecutar la división train/test')

## 8) Predicción y evaluación
Evaluamos la exactitud (accuracy) y mostramos la matriz de confusión y el reporte de clasificación.

In [ ]:
if 'model' in globals() and 'X_test' in globals():
    y_pred = model.predict(X_test)
    print('Accuracy:', accuracy_score(y_test, y_pred))
    print('\nMatriz de confusión:')
    display(confusion_matrix(y_test, y_pred))
    print('\nReporte de clasificación:')
    print(classification_report(y_test, y_pred))
else:
    print('Entrena el modelo primero')

## 9) Importancia de variables
Mostramos qué variables fueron más relevantes para el modelo (según Random Forest).

In [ ]:
if 'model' in globals() and 'X' in globals():
    fi = pd.DataFrame({'feature': X.columns, 'importance': model.feature_importances_}).sort_values(by='importance', ascending=False)
    display(fi.head(20))
    plt.figure(figsize=(8,6))
    sns.barplot(x='importance', y='feature', data=fi.head(15))
    plt.title('Importancia de características (Top 15)')
    plt.tight_layout()
    plt.show()
else:
    print('Necesitamos el modelo entrenado y X definido')

# 10) Ejercicios para que practiquen

1. # TODO: Cambia la columna objetivo por otro nombre si tu dataset usa otro campo.
2. # TODO: Prueba imputar valores nulos en lugar de eliminar filas (por ejemplo con la media).
3. # TODO: Sustituye RandomForestClassifier por LogisticRegression y compara resultados.
4. # TODO: Selecciona un subconjunto de variables (p. ej. 5) y entrena de nuevo. ¿Mejora o empeora la accuracy?
5. Bonus: Usa sklearn.model_selection.GridSearchCV para buscar mejores hiperparámetros para RandomForest.